**1. Collect articles from the blog**

In [4]:
import urllib.request as req
from bs4 import BeautifulSoup

In [5]:
def getLinks(url, links): # collect list of article links
    request = req.Request(url)
    response = req.urlopen(request)
    soup = BeautifulSoup(response)
    for article in soup.findAll('a'):
        url = article['href']
        if url.startswith("https://cakeit.pl/") and "Kategorie" not in url:
            links.append(url)
    return links

Urls = []
articlesUrl = getLinks("https://cakeit.pl/Kategorie/desery/page/2", Urls)
articlesUrl

['https://cakeit.pl/podstawowe-narzedzia-do-tworzenia-skladania-i-tynkowania-tortow/',
 'https://cakeit.pl/poradnik-prezentowy-dla-torciar-i-lubiacych-wypieki/',
 'https://cakeit.pl/najlepsze-formy-silikonowe-do-monoporcji-i-ciast-musowych/',
 'https://cakeit.pl/jak-zaczac-robic-ladne-zdjecia-deserow-radzi-justyna-porowska/',
 'https://cakeit.pl/gadzety-cukiernicze-z-aliexpress-co-warto-kupic/',
 'https://cakeit.pl/podstawowe-narzedzia-do-masy-cukrowej/',
 'https://cakeit.pl/test-mikserow-tani-robot-planetarny-silvercrest-z-lidla-vs-reczny-z-misa/',
 'https://cakeit.pl/jak-najlepiej-zmielic-twarog-test-sernikowy/',
 'https://cakeit.pl/jak-dziala-pieczenie/',
 'https://cakeit.pl/rabaty/',
 'https://cakeit.pl/wspolpraca-i-reklama/',
 'https://cakeit.pl/o-blogu/',
 'https://cakeit.pl/',
 'https://cakeit.pl/podstawowe-narzedzia-do-tworzenia-skladania-i-tynkowania-tortow/',
 'https://cakeit.pl/poradnik-prezentowy-dla-torciar-i-lubiacych-wypieki/',
 'https://cakeit.pl/najlepsze-formy-silikon

In [6]:
def getText(links): # get articles body
    posts = []
    for url in links:
        request = req.Request(url)
        response = req.urlopen(request)
        soup = BeautifulSoup(response)
        posts.append(''.join(map(lambda p: p.text.replace("\n", " "), soup.find_all('article'))))
    return posts

listaTestowa = ["https://cakeit.pl/najlepsze-formy-silikonowe-do-monoporcji-i-ciast-musowych/"]
texting = getText(articlesUrl)
texting

['   Podstawy - tu zacznij, Testy i recenzje, Torty i ozdoby  Podstawowe narzędzia do tworzenia (składania i tynkowania) tortów  Posted on3 czerwca 20203 stycznia 2024AuthorAneta       Obserwuj nas! Nóż, łyżka i tortownica. Tyle potrzebujesz żeby zrobić tort. Jak ten tort ostatecznie będzie wyglądał, to już inna kwestia. 🙂 Jeśli zależy Ci na tym żeby Twoje torty były równe i ładnie otynkowane to przyda Ci się kilka podstawowych narzędzi. Być może nie interesują Cię moje wynurzenia na temat potrzebnych akcesoriów, więc podam Ci krótką listę, z której możesz skorzystać. Szerzej każdy z nich opisze w dalszej części wpisu.  Narzędzia do tortów:* Narzędzia potrzebne do składania tortów:  Rant – do pieczenia biszkoptu i składania tortu Nóż – do krojenia biszkoptu Linijka – do odmierzenia wysokości blatów biszkoptowych Butelka z sitkiem, pędzel lub łyżka – do nasączania biszkoptu Folia rantowa – do łatwiejszego wyciągania tortu z rantu Rękaw cukierniczy – do nakładania kremów  Narzędzia potrz

In [7]:
len(texting)

59

**2. Feature extraction**
Term Frequency or IF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X = vectorizer.fit_transform(texting)
X

<59x6612 sparse matrix of type '<class 'numpy.float64'>'
	with 28288 stored elements in Compressed Sparse Row format>

**3. K-Means algorithm to split articles into grops**

In [26]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)

In [27]:
km.fit(X)

Initialization complete
Iteration 0, inertia 93.10667063270702.
Iteration 1, inertia 47.129435089824995.
Converged at iteration 1: strict convergence.


KMeans(max_iter=100, n_clusters=3, n_init=1, verbose=True)

In [28]:
import numpy as np
np.unique(km.labels_, return_counts=True) # numbers of articles in each cluster 

(array([0, 1, 2]), array([24, 16, 19], dtype=int64))

In [29]:
text = {}
for i, cluster in enumerate(km.labels_): # agregate articles from clusters
    oneDocument = texting[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [30]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [31]:
custom_stop_words = set(stopwords.words('english')+list(punctuation))
keywords = {}
counts = {}
for cluster in range(3):
    word_in_senteces = word_tokenize(text[cluster].lower())
    word_in_senteces = [word for word in word_in_senteces if word not in custom_stop_words]
    frequency = FreqDist(word_in_senteces)
    keywords[cluster] = nlargest(100, frequency, key=frequency.get) # top kewords and each counts
    counts[cluster] = frequency # complete frequency distribution

In [32]:
# keywords unique for each cluster 
unique_keywords = {}
for cluster in range(3):
    other_clusters = list(set(range(3))-set([cluster]))
    keys_other_clusters = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique = set(keywords[cluster]) - keys_other_clusters
    unique_keywords[cluster] = nlargest(10, unique, key=counts[cluster].get)
unique_keywords

{0: ['są', 'zł', 'cena', 'co', 'ok.', 'ci', 'tylko', 'tortu', 'tak', 'jednak'],
 1: ['dodaj',
  'tartę',
  'umieść',
  'tarty',
  'spód',
  'przygotować',
  '2',
  'kremem',
  'cukru',
  'minut'],
 2: ['klikając',
  'forma',
  'ją',
  'formę',
  'je',
  'kształcie',
  'cake',
  'ml',
  'mini',
  'serca']}

**4. K-Nearest Neighbors algorithm to assigne a theme to the article**

In [33]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5)
classifier.fit(X,km.labels_)

KNeighborsClassifier()

In [34]:
article = "Orzechowe tartaletki z gruszką i waniliowym crème pâtissière są idealnym deserem na jesienne dni. Gdybym miała wybrać smaki, które najbardziej kojarzą mi się z jesienią to byłyby to właśnie orzechy i gruszki. Aromatyczne, kruche spody, intensywnie orzechowa pralina, soczyste gruszki oraz waniliowy krem budyniowy to niezwykle smaczne połączenie. artaletki orzech laskowy - gruszka - waniliowy krem budyniowy. Przygotowanie tartaletek możesz bez problemu rozłożyć na kilka dni. Z kilkudniowym wyprzedzeniem możesz przygotować kruche spody (przechowuj w suchym, szczelnie zamykanym pojemniku). Pralinę orzechową (przechowuj w temperaturze pokojowej). Gruszki w syropie (przechowuj w lodówce). Dzień wcześniej przygotuj waniliowy creme patisserie – krem budyniowy (przechowuj w lodówce, zabezpieczony folią spożywczą, tak aby dotykała powierzchni kremu). Deser złóż dopiero w dniu podania. "
test = vectorizer.transform([article])

In [35]:
test

<1x6612 sparse matrix of type '<class 'numpy.float64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [36]:
classifier.predict(test)

array([2])